

# Building a healthcare chatbot with the NVIDIA Haystack integration and PubMed


*notebook by Tilde Thurium:
 [Mastodon](https://tech.lgbt/@annthurium) || [Twitter](https://twitter.com/annthurium) || [LinkedIn](https://www.linkedin.com/in/annthurium/)*


**Prerequisites:**

*   [NVIDIA API Key](https://org.ngc.nvidia.com/setup)

## Installing the NVIDIA-Haystack extension

To start, let's install the latest release of `nvidia-haystack` with `pip`, as well as any other libraries we're going to need:

In [1]:
%%bash

!pip install nvidia-haystack==0.0.1
pip install pymed
pip install git+https://github.com/deepset-ai/haystack-core-integrations.git#subdirectory=integrations/nvidia

  Cloning https://github.com/deepset-ai/haystack-core-integrations.git to /tmp/pip-req-build-o0ujs4mk
  Resolved https://github.com/deepset-ai/haystack-core-integrations.git to commit 7283a5c9009e68cb6199be0100f7d89868b5be8c
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

bash: line 2: !pip: command not found
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack-core-integrations.git /tmp/pip-req-build-o0ujs4mk


Get the credentials we need and set them as an environment variable.

In [2]:
import os
from google.colab import userdata
NVIDIA_API_KEY = userdata.get("NVIDIA_API_KEY")
os.environ['NVIDIA_API_KEY'] = NVIDIA_API_KEY

# PubMed Fetcher

PubMed is the best source of up to date medical research. Now we are going to write our own custom class to pull scientific papers from PubMed that are relevant to the query at hand.

The PubMed sdk basically just wraps the PubMed API so it's easier to query.


In [3]:
from pymed import PubMed
from typing import List
from haystack import component
from haystack.dataclasses import Document


pubmed = PubMed(tool="NvidiaHaystackDemo", email="tilde.thurium@deepset.ai")

def documentize(article):
  print(article.toJSON())
  return Document(content=article.abstract, meta={'title': article.title, 'keywords': article.keywords, 'publication_date': article.publication_date})

@component
class PubMedFetcher():

  @component.output_types(articles=List[Document])
  def run(self, queries: list[str]):
    cleaned_queries = queries[0].strip().split('\n')

    articles = []
    try:
      for query in cleaned_queries:
        response = pubmed.query(query, max_results = 1)
        documents = [documentize(article) for article in response]
        articles.extend(documents)
    except Exception as e:
        print(e)
        print(f"Couldn't fetch articles for queries: {queries}" )
    results = {'articles': articles}
    return results

Instantiate two `NvidiaGenerator`s. One will convert the query into keywords, and the other will actually answer the query based on the documents provided.

In [4]:
from haystack_integrations.components.generators.nvidia import NvidiaGenerator, NvidiaGeneratorModel

llm = NvidiaGenerator(
    model=NvidiaGeneratorModel.NV_LLAMA2_RLHF_70B,
    model_arguments={
        "temperature": 0.2,
        "top_p": 0.7,
        "max_tokens": 1024,
        "seed": None,
        "bad": None,
        "stop": None,
    },
)
llm.warm_up()

keyword_llm = NvidiaGenerator(
    model=NvidiaGeneratorModel.NV_LLAMA2_RLHF_70B,
    model_arguments={
        "temperature": 0.2,
        "top_p": 0.7,
        "max_tokens": 1024,
        "seed": None,
        "bad": None,
        "stop": None,
    },
)
keyword_llm.warm_up()

Next, add tje `PubmedFetcher` into a RAG pipeline.

In [ ]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder

keyword_prompt_template = """
Your task is to convert the following question into 3 keywords that can be used to find relevant medical research papers on PubMed.
Here is an examples:
question: "What are the latest treatments for major depressive disorder?"
keywords:
Antidepressive Agents
Depressive Disorder, Major
Treatment-Resistant depression
---
question: {{ question }}
keywords:
"""

prompt_template = """
Answer the question truthfully based on the given documents.
If the documents don't contain an answer, say so.
cite the documents you used by mentioning their article title in the answer.
For example, begin your answer with ‘As stated in the article titled, ...’.

q: {{ question }}
Articles:
{% for article in articles %}
  {{article.content}}
  keywords: {{article.meta['keywords']}}
  title: {{article.meta['title']}}
{% endfor %}

"""
keyword_prompt_builder = PromptBuilder(template=keyword_prompt_template)

prompt_builder = PromptBuilder(template=prompt_template)
fetcher = PubMedFetcher()

pipe = Pipeline()

pipe.add_component("keyword_prompt_builder", keyword_prompt_builder)
pipe.add_component("keyword_llm", keyword_llm)
pipe.add_component("pubmed_fetcher", fetcher)
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)

pipe.connect("keyword_prompt_builder.prompt", "keyword_llm.prompt")
pipe.connect("keyword_llm.replies", "pubmed_fetcher.queries")

pipe.connect("pubmed_fetcher.articles", "prompt_builder.articles")
pipe.connect("prompt_builder.prompt", "llm.prompt")



While we're at it, let's make an `ask` method to wrap our query fetching. This method makes it easy to pull the query response out of the results.

In [6]:
from IPython.display import display, HTML

def ask(question):
  output = pipe.run(data={"keyword_prompt_builder":{"question":question},
                          "prompt_builder":{"question": question}})
  print(output)



Give it a try!

In [7]:
ask("How are mRNA vaccines being used for cancer treatment?")

{
    "abstract": "Patients with B-cell lymphomas have altered cellular components of vaccine responses due to malignancy and therapy, and the optimal timing of vaccination relative to therapy remains unknown. SARS-CoV-2 vaccines created an opportunity for new insights in vaccine timing because patients were challenged with a novel antigen across multiple phases of treatment. We studied serologic mRNA vaccine response in retrospective and prospective cohorts with lymphoma and CLL, paired with clinical and research immune parameters. Reduced serologic response was observed more frequently during active therapies, but non-response was also common within observation and post-treatment groups. Total IgA and IgM correlated with successful vaccine response. In individuals treated with CART-19, non-response was associated with reduced B and T follicular helper cells. Predictors of vaccine response varied by disease and therapeutic group, and therefore further studies of immune health during a

**What's next**

In this tutorial, you learned to make a basic chatbot using a custom Haystack Retriever that pulls data from PubMed. In order to make this chatbot fancier, you could make improvements like:


*   Adding additional data sources
*   Giving the bot a name or personality
*   Making it a [stateful agent so it remembers past queries](https://docs.haystack.deepset.ai/docs/agent#conversational-agent-memory)

To see what else is possible with Haystack, you can [browse these tutorials](https://haystack.deepset.ai/tutorials) or check us out on [GitHub](https://github.com/deepset-ai/haystack). Thanks for reading!




